In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

In [2]:
# Pkg.add("NamedTupleTools")

In [1]:
using NamedTupleTools
using StaticArrays
using BenchmarkTools

## @show macro

In [3]:
let

x::Float64 = 64.0

function myshow(x::Float64)
    println("x= " , x);
end

myshow(x) # custom  function 

@show x #macro

# show generated code
Base.remove_linenums!(@macroexpand @show x) 
end

x= 64.0
x = 64.0


quote
    Base.println("x = ", Base.repr(begin
                local var"#30#value" = x
            end))
    var"#30#value"
end

 ## Expression tree (computational graph)

In [4]:
ex = :(x + y * z)

:(x + y * z)

In [5]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol y
        3: Symbol z


In [6]:
using TreeView

@tree x + y * z

LoadError: ArgumentError: Package TreeView not found in current path.
- Run `import Pkg; Pkg.add("TreeView")` to install the TreeView package.

# Creating Macros

In [207]:
macro sayhello(str)
    return :( println($str) )
end

@sayhello "selv"

selv


# Code generation

In [86]:
struct SampleStruct
    x::Float64
    y::Float64
end

obj = SampleStruct(1., 2.)

SampleStruct(1.0, 2.0)

In [87]:
for op = (:sin, :cos, :tan, :log, :exp)
    eval(quote
        Base.$op(a::SampleStruct) = [$op(a.x) $op(a.y)]
    end)
end

sin(obj)

1×2 Matrix{Float64}:
 0.841471  0.909297

# Fieldvector

In [5]:
struct Point3D <: FieldVector{3, Float64}
    x::Float64
    y::Float64
    z::Float64
end


In [621]:
point = Point3D(1.,2.,3.);

In [664]:
waste(p) =  SVector{3}(p)


waste (generic function with 1 method)

In [669]:
@time waste(point)

  0.000004 seconds (1 allocation: 32 bytes)


3-element SVector{3, Float64} with indices SOneTo(3):
 1.0
 2.0
 3.0

### Struct to Tuple

### Struct to array

In [630]:
@time waste(point)

  0.000005 seconds (1 allocation: 80 bytes)


3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [63]:
@time  A = @SMatrix rand(3,3)

function mult(A, point)
    point2 = A * point
    return point2
end

  0.000002 seconds (1 allocation: 80 bytes)


mult (generic function with 3 methods)

In [65]:
@time mult(A, point)

  0.000005 seconds (1 allocation: 32 bytes)


3-element Point3D with indices SOneTo(3):
 3.2656765334139397
 2.114843778462256
 3.059474147191788

### Static array to struct

In [85]:
@time convert(Point3D, @SVector[1.,2., 3.])

  0.000001 seconds


3-element Point3D with indices SOneTo(3):
 1.0
 2.0
 3.0

### view to struct 

In [345]:
mutable struct Point3DMutable <: FieldVector{3, Float64}
    x::Float64
    y::Float64
    z::Float64
end

mutable struct Quad2DControlCmd1 <: FieldVector{2,Float64}
    body_thrust::Float64
    body_torque::Float64
end

Quad2DControlCmd = Quad2DControlCmd1

Quad2DControlCmd1

In [579]:
function selv1(d_state, state, params, t)
    nx = 3
    #X = @view state[1:nx]
    
    # state = convert(Point3DMutable, X)
    return @SVector[1,2]
end

selv1 (generic function with 1 method)

In [582]:
state = rand(8)
quad_2d_params = (; m=1.0, L=0.1, I_xx=0.003)

@time selv1(state,state, quad_2d_params, 0.1)



  0.000003 seconds


2-element SVector{2, Int64} with indices SOneTo(2):
 1
 2

# Waste

In [469]:
function selv(nx) 
    return @SVector[1,2]
end

selv (generic function with 5 methods)

In [470]:
@time selv(3);

  0.000001 seconds
